#__Case Tecnico - Data Scientist__

##__Instruções__

Certifique-se de preenchê-lo e enviá-lo dentro do prazo mencionado em nosso e-mail.
Este teste foi elaborado para avaliar suas habilidades técnicas e seu pensamento crítico. Responda da forma mais organizada e completa possível. Depois de concluir o teste, preencha o Formulário do Google fornecido no e-mail e anexe o arquivo .ipynb com as respostas ao formulário.

__Complete as perguntas neste Jupyter Notebook (arquivo .ipynb) e salve seu trabalho em uma nova versão do arquivo__. Quando terminar, envie-nos o notebook atualizado para análise, anexando-o ao Google Forms enviado em nosso e-mail. Se você usar imagens para resolver qualquer uma das perguntas abaixo, não se esqueça de anexá-las ao Google Forms.

Para otimizar a visualização da estrutura do arquivo, é recomendável usar o [Google Colab](https://colab.research.google.com/). <br>.

Por fim, __não se esqueça de nomear seu arquivo .ipynb__ com as respostas seguindo o exemplo abaixo:<br>
Meu_Nome_Posição_Nivel_Senioridade <br>
Exemplo: CaioGarvil_Data Engineer_Junior.ipynb


##__Problema 1: Problema do stockout__

Uma rede de supermercados sofre com a constante falta de alguns produtos, mais especificamente cerveja. Quando os funcionários notam a falta dos produtos em estoque, eles fazem um pedido à matriz para que enviem mais unidades do centro de distribuição para a loja, mas os caminhões podem demorar até 3 dias para chegar a seu destino, fazendo com que a loja perca várias vendas. O cliente pede que um modelo preditivo seja construído, a fim de que cada loja possa saber, com antecedência, quando haverá uma falta de estoque. Eles mencionam ter uma tabela de histórico de ocorrência de stockout por loja, produto e dia, além de algumas outras informações que podem ser úteis (mas não especificam quais).

### A.  Que tipos de dados você perguntaria se eles têm e qual granularidade você pediria? Ou que tipo de informações externas você buscaria? Explique o racional por trás de suas escolhas.

#### -------- ANSWER PROBLEM 1.A. -------- 

Since the client already provides a table of historical stockout occurrences by store, product, and day, the first step is to leverage this outcome variable as the target of the predictive model. In other words, the stockout flag itself can serve as the dependent variable, while the challenge becomes engineering features that anticipate it before it occurs. However, a simple predictive model may not be sufficient, because stockouts can arise from two fundamentally different mechanisms: an unexpected surge in demand or a delay in supply. Distinguishing between these drivers is essential, as the corrective actions differ: in one case the store needs better demand forecasting, in the other it needs greater reliability from the replenishment system.

To address this, I propose a two-component system that explicitly models both demand and supply chain dynamics, and then integrates them through inventory accounting in order to classify and predict not only the likelihood but also the cause and timing of future stockouts.

##### -------- Goal -------- 

Build and validate a two-component system: <u>a baseline demand model</u> that predicts expected sales and generates prediction intervals at the store–SKU–day level of granularity; and a <u>supply/lead-time model</u> combined with inventory accounting to simulate stock trajectories.

#####  -------- Pipeline --------
  
- 1) Data setup & leakage controls  
  - Build a panel: store//SKU//day with sales (units), on-hand, receipts (qty, timestamps), orders (order/ship/deliver), price/promo, calendar, weather/events. 
  - Train/validation: rolling-origin time series cross-validation (5 folds). Prohibit look-ahead (no future deliveries/prices). Optional holdout of stores/SKUs for generalization.  

- 2) Feature engineering & target framing  
  - Lags & trends: sales lags (1/3/7), moving averages (7/14), weekday/holiday seasonality.  
  - Price/promo signals: % discount, promo binary, price elasticity features.  
  - Inventory signals: days of coverage = on-hand / MA(sales), days since last delivery, on-order qty.  
  - **Target(s):**  
    - For demand model: sales (units).  
    - For classification later: stockout flag at D+1..D+3 (derived after simulation).  

- 3) Baseline demand model  
  - Start with ARIMAX / regularized GLM (Poisson/NegBin with store×SKU fixed effects).  
  - Alternatives: Gradient Boosted Trees (XGBoost/LightGBM) or state-space if needed.  
  - Output per (store, SKU, day): point forecast \(\hat{y}_t\).  

- 4) Prediction intervals (uncertainty)  
  - Option A: model-based variance (GLM/ARIMAX).  
  - Option B: quantile models (e.g., Q10/Q90).  
  - Option C: conformal prediction on residuals (distribution-free).  
  - Output: \(PI^{low}_t, PI^{high}_t\) per day.  

- 5) Lead-time (supply) model  
  - Compute \(LT = \text{delivery} - \text{ship}\) by route/CD/weekday/season.  
  - Fit empirical or parametric distribution (Log-Normal/Gamma).  
  - Output: expected lead time \(E[LT|\cdot]\) and percentiles (P80/P90); \(P(\text{arrive by } t)\) for open POs.  

- 6) Inventory accounting (projection)  
  - Daily recursion: \(\text{Inv}_t = \text{Inv}_{t-1} - \hat{y}_t + \sum \text{deliveries}_t\).  
  - Stochastic option: simulate paths using sales PI and lead-time uncertainty.  
  - Output: projected inventory and **stockout probability** for D+1..D+3.  

- 7) Root-cause attribution logic  
  - **Demand-driven** if actual sales \(> PI^{high}\) (or projected breach drives depletion) and deliveries on-time.  
  - **Supply-driven** if sales within PI but expected delivery misses (realized \(LT\) > P90 or \(P(\text{arrive by } t)\) high yet no arrival).  
  - **Mixed** if both hold; **Residual** otherwise. Thresholds tuned in validation.  

- 8) Backtesting protocol  
  - Rolling folds: for each fold, refit demand & lead-time models, project inventory, record stockout predictions and attributions.  
  - Build weak labels from history (late PO vs demand spike) to score attribution.  

- 9) Evaluation metrics  
  - Demand forecast: sMAPE/MAPE/RMSE; PI coverage (e.g., 90% nominal ≈ 90% empirical).  
  - Stockout prediction (D+1..D+3): Precision/Recall/F1; average days of advance notice.  
  - Attribution (Demand vs Supply): class-wise Precision/Recall/F1 + confusion matrix.  

- 10) Robustness & ablations  
  - No-externals vs full (weather/events) uplift; with/without FE; promo vs no-promo weeks; heat/rain shocks.  
  - Censoring check (sales capped by zero stock): optional Tobit/latent-demand correction.  
  - Cold-start: hierarchical shrinkage (brand/store groups) or similarity pooling.  

- 11) Decision layer (ops-ready outputs)  
  - Daily scores per store–SKU: \(P(\text{stockout in } D+1..D+3)\), root-cause flag (Demand/Supply/Mixed), recommended reorder if coverage < expected \(LT\) + safety days.  
  - Simple policy table with thresholds derived from backtests (optimize service level vs false alarms).  

- 12) Deliverables (for the PS notebook)  
  - One example SKU/store chart (6–8 weeks): actual vs forecast + PI, expected vs realized deliveries, inventory path, attributed cause at stockout.  
  - Metrics table (by fold and overall).  
  - Brief discussion of trade-offs and next steps (e.g., integrate real-time PO status).





### B. Escolha um dos problemas abaixo, relacionados a dados de séries temporais, e sugira uma implementação para resolver ou mitigar o problema:

    *   Algumas das features (por exemplo, vendas_diarias) tem muitos outliers
    *   Algumas lojas e produtos têm pouca quantidade de dados
    *   Série temporal de informação de stockout são muito esparsas (as séries  são compostas majoritariamente por valores nulos, com alguns valores não-nulos espalhados de maneira irregular)





ESCREVA SUA RESPOSTA AQUI

##__Problema 2: Problema de turnover__

Você trabalha em uma empresa de produtos alimentícios que sofre com grande taxa de turnover (ou seja, funcionários que deixam a empresa). O departamento de RH está buscando entender o que leva seus funcionários a deixarem a empresa ou não.

Assuma que a empresa possui um dataset com as seguintes colunas e aproximadamente 5 anos de dados:

    *  employee_id: Identificação única do funcionário.

    *  manager_id: ID do gerente do funcionário

    *  employee_name: Nome do funcionário

    *  age: Idade do funcionário

    *  gender: Gênero do funcionário

    *  food_preferences: Preferências alimentares registradas para eventos corporativos

    *  social_media_handle: Identificadores de redes sociais utilizadas pelo funcionário

    *  job_level: Nível do cargo ocupado (de 1 a 5)

    *  monthly_income: Salário mensal.

    *  years_at_company: Anos trabalhados na empresa.

    *  distance_from_home: Distância da casa até o trabalho (em km)

    *  over_time: Se o funcionário faz horas extras (Sim ou Não).

    *  company_car_assigned: Informação sobre se o funcionário tem um carro da empresa

    *  turnover: 1 se o funcionário deixou a empresa, 0 se não deixou

### A. Desenvolva uma estratégia para prever a variável turnover. Quais etapas esse projeto deveria ter, desde a coleta de dados até a disponibilização dos resultados? (Descreva bem as etapas e as atividades em cada uma delas)

ESCREVA SUA RESPOSTA AQUI

### B. Que tipo de algoritmo você aplicaria/testaria e por quê?

ESCREVA SUA RESPOSTA AQUI

### C. Agora considere que, por questões burocráticas, o projeto teve que ser reduzido e você terá apenas metade do tempo para executá-lo. Que atividades você cortaria ou reduziria e por quê?

ESCREVA SUA RESPOSTA AQUI

##__Problema 3: Análise de regressão__

Uma empresa da indústria farmacêutica tem uma nova política interna para não discriminar significativamente os salários de acordo com a localização de seus funcionários. Eles coletaram os dados de todos os funcionários e querem que você verifique se eles já estão seguindo a nova política. Antes de responder às perguntas abaixo, dê uma olhada no conjunto de dados em anexo: (1stPhase-SelectiveProcess-Data Science-Data Base.csv)

### A.  Descreva como você pode usar os dados do supermercado para verificar se os funcionários de diferentes locais têm salários significativamente diferentes. (Inclua aqui como você tratará as variáveis antes de inseri-las no modelo e todas as etapas que pretende seguir aprofundando o máximo possível em sua explicação)

ESCREVA SUA RESPOSTA AQUI

### B. Imagine que, depois de olhar os dados, você tenha uma reunião com o cliente em que poderá mostrar alguns gráficos e fazer comentários e/ou perguntas para ele sobre os dados. Quais visuais você levaria e quais comentários/perguntas você faria?

ESCREVA SUA RESPOSTA AQUI

### C.  Implemente a sua abordagem da melhor forma possível em um código  Python, comentando as etapas do seu código. Não esqueça de trazer uma conclusão final.

In [ ]:
# ESCREVA SEU CODIGO AQUI